# Joint Intent detection and slot filling
Dieses Jupyter notebook wurde als semesterabschließende Arbeit für das Modul Natural Language Processing an der [Fachhochschule Südwestfalen](https://www.fh-swf.de/en/international_3/index.php) erstellt.

## Einleitung
Das Joint intent detection and slot filling (IDSF) ist eine Aufgabe aus dem Teilbereich des Natural Language Understandings (NLU) des Natural Language Processings (NLP), die uns in heutzutage fast täglich Alltag begegnet. Sei es um einen Timer auf dem Handy zu starten, bestimmte Musik abzuspielen oder das Licht einzuschalten. Der Ablauf ist dabei häufig der selbe: "Siri stelle einen Timer für 4 Minuten", "Alexa spiele meine Schlager Playlist" oder "Google erstelle einen Arzttermin für heute 16:00 Uhr". Meist beginnen die Kommandos mit dem Namen des Sprachassistenten, um diesen zu aktivieren, gefolgt vom Kommando für die gewünschte Aktion. Das IDSF beschäftigt sich dabei mit der Aufgabe, die gewünschte Aktion (Intent), also stelle einen Timer, Spiele Musik, erstelle einen Termin im Kalender und die dazugehörigen notwendigen Parameter, wie z.B. vier Minuten, Schlager Playlist oder Arzt heute 16:00 Uhr (Slots) zu erkennen.
Da der Gebrauch dieser Sprachassistenten in Zukunft wahrscheinlich noch stärker zu nehmen wird, wollen wir uns deren funktionsweise in diesem Notebook näher anschauen. Dafür wird zuerst die Entwicklungshistorie vom IDSF betrachtet und anschließend wird ein eigenes Modell für die Erkennung erstellt und anhand eines selbst vorbereiteten Korpus trainiert.

## Joint Intent Detection and Slot filling

--------------

## Datenbeschaffung

Als Datensatz für das nachfolgende Beispiel verwenden wir den Snips-Datensatz. Dieser Datensatz wurde vom, mittlerweil zu Sonos gehörenden [1], [Snips Team](https://snips.ai/) zusammengestellt, um ihr eigenes Modell mit anderen Wettbewerbern wie zum Beispiel Amazons Alexa zu verglichen. Die Ergebnisse und die Datensätze der drei Vergleiche wurden in einem [GitHub Repository](https://github.com/sonos/nlu-benchmark/tree/master) veröffentlicht und in dem Paper [Coucke A. et al., "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces." 2018](https://arxiv.org/abs/1805.10190) erläutert. Das Repository enthält die Daten für drei Evaluationen aus den Jahren 2016 bis 2018. Wir werden in diesem Notebook die Daten der 2017 durchgeführten Evaluation verwenden, da diese Sätze für sieben unterschiedliche und allgemeine Aufgaben enthält.

Quellen
1: https://investors.sonos.com/news-and-events/investor-news/latest-news/2019/Sonos-Announces-Acquisition-of-Snips/default.aspx, [Online, 07.03.2025]

## Erstellen eines Corpus